#Introduction

Prior to the 2013-2014 NBA season, SportVu tracking cameras were installed in all 30 NBA arenas. These six cameras would use cutting edge technology to track the exact location of all players, ref and the ball 25 times a second. This tracking data could later be used by teams, the league and privately run companies to provide new insights and analysis about the game of basketball.

Although most of this data is heavily guarded, a small portion is available publicly on [NBA.com](http://stats.nba.com/tracking/). It is our hope to use this data, combining it with the previously available play-by-play and shot chart data, to create a model that will give us the probability that a given shot will go in. With this model we will explore many insights that could prove valuable to fans and teams alike.

In this notebook, we go throughout the process of using the NBA.com API to gather the data we need. We then combine it all, a process that is not easy as one dataset is indexed differently than the others. In terms of time to run, this notebook takes by far the most time, so run it at your peril!

#Getting the Data

In [2]:
## Import a bunch of modules that may be helpful
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import requests
!pip install nbastats
import nbastats.nbastats as nbastats
!pip install py-goldsberry
import goldsberry

The first type of data that we want to get is the SportVu tracking data for all shots. These shots are sorted by player and are located on the ['Shot Log' page of their player profile](http://stats.nba.com/player/#!/201939/tracking/shotslogs/). So in order to get all the shots in a given year, we first need to get a list of players who were active in that year.

In [3]:
# get players in season a
def players(a):
    url = "http://stats.nba.com/stats/commonallplayers?"
    api_param = {'IsOnlyCurrentSeason':"0",
                          'LeagueID': '00',
                          'Season': '2015-16'}
    pull = requests.get(url, params=api_param)
    _headers = pull.json()['resultSets'][0]['headers']
    _values = pull.json()['resultSets'][0]['rowSet']
    season=pd.DataFrame([dict(zip(_headers, value)) for value in _values])
    season = season.query("FROM_YEAR <= @a and TO_YEAR >= @a")
    return season

In [4]:
players2014=players('2014')
players2013=players('2013')

Other data that we want that is not already included with each player or each shot is the height of the player who is shooting it, along with the height of the closest defender. We will therefor also scrape the height of all the players, when available.

In [5]:
height_2013 = []
for indx,row in players2013.iterrows():
    height_2013.append(goldsberry.player.demographics(row['PERSON_ID']).player_info()[0]['HEIGHT'])
players2013['HEIGHT'] = height_2013

In [8]:
height_2014 = []
for indx,row in players2014.iterrows():
    height_2014.append(goldsberry.player.demographics(row['PERSON_ID']).player_info()[0]['HEIGHT'])
players2014['HEIGHT'] = height_2014   

Using this list of players in the two years for which we want data, we can write a function that gets a shot log of player for a given year, given his ID and year.

In [ ]:
## function for getting the shot log of a player given his ID and the year
def shot_log(year,player_id):
    api_param = {'DateFrom' : '',
                    'DateTo' : '',
                    'GameSegment': '',
                    'LastNGames': 0,
                    'LeagueID': "00",
                    'Location': '',
                    'Month': 0,
                    'OpponentTeamID': 0,
                    'Outcome': '',
                    'Period': 0,
                    'PlayerID': player_id,
                    'Season': year,
                    'SeasonSegment': '',
                    'SeasonType': "Regular Season",
                    'TeamID': 0,
                    'VsConference':'',
                    'VsDivision': ''
                               }
    pull=requests.get("http://stats.nba.com/stats/playerdashptshotlog?",params=api_param)
    headers = pull.json()['resultSets'][0]['headers']
    values = pull.json()['resultSets'][0]['rowSet']
    shots=pd.DataFrame([dict(zip(headers, value)) for value in values])
    return shots


We can then run this function all all players for both years. Notice that in this loop we are also using a function in one of the modules we imported to get the shot chart data for each shot. This provides us with new information like the type of shot, general area (Right, Left, Middle) of each shot, and x,y coordinates.

In [293]:
#Get shot chart and shot log for all players in 2014
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in players2014.iterrows():
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2014-15")
    all_chart=all_chart.append(chart.shotchart())
    shots = shot_log("2014-15",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
all_shots.to_csv("all_shots_2014.csv")
all_chart.to_csv("all_chart_2014.csv")

In [295]:
#Get shot chart and shot log for all players in 2013
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in season2013.iterrows():
    shots = shot_log("2013-14",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2013-14")
    all_chart=all_chart.append(chart.shotchart())
all_shots.to_csv("all_shots_2013.csv")
all_chart.to_csv("all_chart_2013.csv")

We now need to merge these two data sources. What is annoying is that there is no common key for the shots, and moreover the game time at which each shot occurs varies across the datasets, as one is recorded by a computer and the other by hand. Another very annoying trait that occured is sometimes one dataset would say a shot occured while the other had no reference of it. Therefor, the process I used is to group the shots by the player who was shooting, and then group by game and quarter. If the amounts of shots were the same I would then merged the datasets ordered by game clock, the idea being that if one shot was before another in one dataset it would be before the other in the other data set. If the the amounts of shots were not the same I would then go through the shots and find the shot that occured closest to it - if they were within five seconds of eachother I would merge them, if not I would not merge them and drop that observation.

In [ ]:
## Merging shot chart data with player tracking for 2014
## DO NOT RUN... takes a while
def merge_shot_data(all_shots,all_chart):
    players=set(all_shots['PLAYER_ID'])
    joined_shots=pd.DataFrame()
    joined_shots_1=pd.DataFrame()
    for player in players:
        shots = all_shots.loc[all_shots.PLAYER_ID==player].reset_index(drop=True)
        chart = all_chart.loc[all_chart.PLAYER_ID==player].reset_index(drop=True)
        split_time = shots.GAME_CLOCK.str.split(':')
        col_names=[u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', 
                   u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', 
                   u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', 
                   u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W',u'GRID_TYPE',u'GAME_EVENT_ID',
                   u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',u'MINUTES_REMAINING', u'SECONDS_REMAINING', 
                   u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', 
                   u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', 
                   u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']
        time = pd.DataFrame(shots.GAME_CLOCK.str.split(':').tolist(), columns="MIN SECONDS".split())
        shots['MIN'] = time['MIN']
        shots['SECONDS'] = time['SECONDS']
        shots['TIME']=60*shots.MIN.astype(int)+shots.SECONDS.astype(int)
        chart['TIME']=60*chart.MINUTES_REMAINING+chart.SECONDS_REMAINING
        games = set(chart["GAME_ID"])
        for game in games:
            game_shots_chart=chart.loc[chart.GAME_ID==game].reset_index(drop=True)
            game_shots_log = shots.loc[shots.GAME_ID==game].reset_index(drop=True)
            if game_shots_chart.shape[0]==game_shots_log.shape[0]:
                huh = pd.concat([game_shots_chart,game_shots_log],axis=1)
                huh=huh[col_names]
                joined_shots=joined_shots.append(huh)
            else:
                quarters=set(game_shots_chart['PERIOD'])
                for quarter in quarters:
                    same_period=game_shots_log.loc[(game_shots_log.PERIOD==quarter)].reset_index(drop=True)
                    same_period_chart=game_shots_chart.loc[(game_shots_chart.PERIOD==quarter)].reset_index(drop=True)
                    if (not same_period.empty) and (not same_period_chart.empty):
                        if same_period_chart.shape[0]==same_period.shape[0]:
                            huh = pd.concat([same_period_chart,same_period],axis=1)
                            huh=huh[col_names]
                            joined_shots=joined_shots.append(huh)
                        else:
                            for index,row in same_period_chart.iterrows():
                                same_period['DIF']=abs(same_period.TIME-row.TIME)
                                df=same_period.sort(['DIF'],ascending=True)
                                same_time=same_period.loc[[0]]
                                if same_time.DIF[0]<5:
                                    a=same_period_chart.loc[[index]].reset_index(drop=True)
                                    huh=pd.concat([a,same_time],axis=1)
                                    huh=huh[col_names]
                                    joined_shots_1=joined_shots_1.append(huh,ignore_index=True)
        print player
    return joined_shots
all_shots = pd.read_csv("all_shots_2013.csv")
all_chart = pd.read_csv("all_chart_2013.csv")
joined_shots = merge_shot_data(all_shots,all_chart)
joined_shots.to_csv("joined_shots_2013.csv")
all_shots = pd.read_csv("all_shots_2014.csv")
all_chart = pd.read_csv("all_chart_2014.csv")
joined_shots = merge_shot_data(all_shots,all_chart)
joined_shots.to_csv("joined_shots_2014.csv")

Wow, that took a while! Now what we want to do is get the height of the shooter and closest defender, when available. To maximize the speed of this we can get a subset that belongs to each player (first on offense, then defense) and then set that whole column equal to the height. Doing so for 2013...

In [9]:
all2013=pd.read_csv('joined_shots_2013.csv')
all2013['off_height']=0
all2013['def_height']=0
players_shot=list(set(all2013['PLAYER_ID']))
players_def=list(set(all2013['CLOSEST_DEFENDER_PLAYER_ID']))
for player in players_shot:
    at = all2013[all2013['PLAYER_ID']==player].index.tolist()
    hi = players2013.loc[players2013.PERSON_ID==player]['HEIGHT']
    all2013['off_height'][at]=hi.iloc[0]
for player in players_def:
    at = all2013[all2013['CLOSEST_DEFENDER_PLAYER_ID']==player].index.tolist()
    hi = players2013.loc[players2013.PERSON_ID==player]['HEIGHT']
    all2013['def_height'][at]=hi.iloc[0]

And now doing so for 2014...

In [10]:
all2014=pd.read_csv('joined_shots_2014.csv')
all2014['off_height']=0
all2014['def_height']=0
players_shot=list(set(all2014['PLAYER_ID']))
players_def=list(set(all2014['CLOSEST_DEFENDER_PLAYER_ID']))
for player in players_shot:
    at = all2014[all2014['PLAYER_ID']==player].index.tolist()
    hi = players2014.loc[players2014.PERSON_ID==player]['HEIGHT']
    all2014['off_height'][at]=hi.iloc[0]
for player in players_def:
    at = all2014[all2014['CLOSEST_DEFENDER_PLAYER_ID']==player].index.tolist()
    hi = players2014.loc[players2014.PERSON_ID==player]['HEIGHT']
    if len(hi)>0:
        all2014['def_height'][at]=hi.iloc[0]

Now we need our last piece of data, play by play. This dataset will provide us with other key pieces of information we need for our insights that we don't already have, like score differential at time of shot, whether the action before was a turnover (to identify fast break opportunities) and the ids of the players who are on the court.

For this we will borrow heavily from the work of Darryl Blackport, who made his iPython Notebook on how to get play by play along with the players who are on the court at each moment [available online](http://nbviewer.ipython.org/github/dblackrun/nba/blob/master/ipython_notebooks/pbp_with_players_on_court.ipynb). However, I did have to add in some code, as his code was only for one specific game and when using it in a loop over all games we noticed it broke down in some instances. I will highlight those occations when we get to them

In [11]:
import json
import urllib2
import pandas as pd
import numpy as np
import urllib2
import math
import csv
# base url for play by play for game id
GAME_BASE_URL = "http://stats.nba.com/stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID=<game_id>&RangeType=2&Season=2014-15&SeasonType=Regular+Season&StartPeriod=1&StartRange=0"
# base url for moment data for event id
MOMENT_BASE_URL = "http://stats.nba.com/stats/locations_getmoments/?eventid=<event_id>&gameid="

In [4]:
def getRawPbpForGame(game_id):
    # for a given game_id, return a pandas data frame with the raw play by play
    url = GAME_BASE_URL.replace("<game_id>", game_id)
        
    response = urllib2.urlopen(url)
    data = json.loads(response.read())

    game_info = []
    plays = []
    for line in data['resultSets']:
        if 'name' in line.keys() and line['name'] == 'PlayByPlay':
            for event in line['rowSet']:
                row = dict(zip([header for header in line['headers']],event))
                plays.append(row)

    return pd.DataFrame(plays)

The below code is what he used to get the IDs of the players on the court at a given moment.

In [ ]:
def getPlayersOnFloorForMoment(game_id, event_id):
    # for a given game_id and event_id, return a dict with a list players on the floor for each team and team ids
    url = MOMENT_BASE_URL.replace("<event_id>", str(event_id))
    url = url + game_id
    response = urllib2.urlopen(url)
    data = json.loads(response.read())
    players = {}
    players['home_team_id'] = data["moments"][0][5][1][0]
    players['away_team_id'] = data["moments"][0][5][6][0]
    players['home_player_ids'] =[]
    players['away_player_ids'] =[]
    for i in range(1,6):
        players['home_player_ids'].append(data["moments"][0][5][i][1])
    for i in range(6,11):
        players['away_player_ids'].append(data["moments"][0][5][i][1])
    return players

We had to edit it to be the following:

We had to do this because for some moments in some games, there is not data at all. For clarification, each moment is a play, so a few seconds, and contains each observation of the players' positions 25 times seconds, so we are checking that data for the players' IDs. Therefor, if that was the case, we had to catch that error. If there was data there was also the case that some of the data could be missing - i.e. for the first of the datapoints about where the players were only 9 players were listed instead of 10. If that was the case then we had to go onto the next data point and see if there was complete data for that. If there was a time period with no moments (this usually happened to entire games, and I'm assuming because the cameras were turned off) then we just filled the moments with NaNs.

In [5]:
def getPlayersOnFloorForMoment(game_id, event_id):
    # for a given game_id and event_id, return a dict with a list players on the floor for each team and team ids
    url = MOMENT_BASE_URL.replace("<event_id>", str(event_id))
    url = url + game_id
    players = {}
    try: 
        r = urllib2.urlopen(url)
    except urllib2.URLError, e:
        r = e
    if(r.code==200):
        response = urllib2.urlopen(url)
        data = json.loads(response.read())
        if data["moments"]:
            huh=0
            found=0
            addit=0
            while found==0:
                if len(data["moments"][huh][5])==11:
                    found =1
                elif (len(data["moments"][huh][5])==10):
                    if (data["moments"][huh][5][0][1]>10):
                        found =1
                        addit=-1
                    elif huh ==(len(data["moments"])-1):
                        found = 2
                    else:
                        huh=huh+1
                elif huh ==(len(data["moments"])-1):
                    found = 2
                else:
                    huh=huh+1
            if found==1:
                players['home_team_id'] = data["moments"][huh][5][1][0]
                players['away_team_id'] = data["moments"][huh][5][6][0]
                players['home_player_ids'] =[]
                players['away_player_ids'] =[]
                for i in range(1,6):
                    players['home_player_ids'].append(data["moments"][huh][5][i+addit][1])
                for i in range(6,11):
                    players['away_player_ids'].append(data["moments"][huh][5][i+addit][1])
            elif found ==2:
                players['home_team_id'] = np.nan
                players['away_team_id'] = np.nan
                players['home_player_ids'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
                players['away_player_ids'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
        else:
            players['home_team_id'] = np.nan
            players['away_team_id'] = np.nan
            players['home_player_ids'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
            players['away_player_ids'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
    else:
            players['home_team_id'] = np.nan
            players['away_team_id'] = np.nan
            players['home_player_ids'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
            players['away_player_ids'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
    return players

In Blackport's code he got the list of players on the court for the entire period by getting it for the first action and then keeping track of substituions. His code was:

In [7]:
def getPlayersOnFloorForPeriod(period):
    # for a given period data frame, return a data frame with new columns for the players on the floor
    period = period.reset_index(drop=True)
    start_event_num = period['EVENTNUM'].min()
    period_number = period['PERIOD'].mean()
    if period_number == 1 or period_number == 3 or period_number > 4:
        start_event_num += 1
    period_starters = getPlayersOnFloorForMoment(game_id, start_event_num)
    period['HOME_TEAM_ID'] = period_starters["home_team_id"]
    period['AWAY_TEAM_ID'] = period_starters["away_team_id"]
    period['HOME_PLAYER1_ID'] = period_starters['home_player_ids'][0]
    period['HOME_PLAYER2_ID'] = period_starters['home_player_ids'][1]
    period['HOME_PLAYER3_ID'] = period_starters['home_player_ids'][2]
    period['HOME_PLAYER4_ID'] = period_starters['home_player_ids'][3]
    period['HOME_PLAYER5_ID'] = period_starters['home_player_ids'][4]
    period['AWAY_PLAYER1_ID'] = period_starters['away_player_ids'][0]
    period['AWAY_PLAYER2_ID'] = period_starters['away_player_ids'][1]
    period['AWAY_PLAYER3_ID'] = period_starters['away_player_ids'][2]
    period['AWAY_PLAYER4_ID'] = period_starters['away_player_ids'][3]
    period['AWAY_PLAYER5_ID'] = period_starters['away_player_ids'][4]
    
    # get index for all substitutions and for each one sub in and out appropriate players
    subs = period[period['EVENTMSGTYPE'] == 8].index.tolist()
    end = len(period.index)
    for i in range(len(subs)):
        if str(period['HOME_PLAYER1_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'HOME_PLAYER1_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['HOME_PLAYER2_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'HOME_PLAYER2_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['HOME_PLAYER3_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'HOME_PLAYER3_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['HOME_PLAYER4_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'HOME_PLAYER4_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['HOME_PLAYER5_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'HOME_PLAYER5_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['AWAY_PLAYER1_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'AWAY_PLAYER1_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['AWAY_PLAYER2_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'AWAY_PLAYER2_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['AWAY_PLAYER3_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'AWAY_PLAYER3_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['AWAY_PLAYER4_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'AWAY_PLAYER4_ID'] = str(period['PLAYER2_ID'][subs[i]])
        elif str(period['AWAY_PLAYER5_ID'].iloc[subs[i]]) == str(period['PLAYER1_ID'][subs[i]]):
            period.ix[subs[i]:end, 'AWAY_PLAYER5_ID'] = str(period['PLAYER2_ID'][subs[i]])
    return period

We keep it mostly the same, but for the instances where we could not get data for the first moment of the period we tried the next five. This is our code (note the while loop).

In [8]:
def getPlayersOnFloorForPeriod(period):
    # for a given period data frame, return a data frame with new columns for the players on the floor
    period = period.reset_index(drop=True)
    start_event_num = period['EVENTNUM'].min()
    period_number = period['PERIOD'].mean()
    if period_number == 1 or period_number == 3 or period_number > 4:
        start_event_num += 1
    period_starters = getPlayersOnFloorForMoment(game_id, start_event_num)
    tries=0
    while (math.isnan(period_starters['home_team_id']))&(tries<5):
        start_event_num=start_event_num+1   
        period_starters = getPlayersOnFloorForMoment(game_id, start_event_num)
        tries=tries+1
    period['HOME_TEAM_ID'] = period_starters["home_team_id"]
    period['AWAY_TEAM_ID'] = period_starters["away_team_id"]
    period['HOME_PLAYER1_ID'] = period_starters['home_player_ids'][0]
    period['HOME_PLAYER2_ID'] = period_starters['home_player_ids'][1]
    period['HOME_PLAYER3_ID'] = period_starters['home_player_ids'][2]
    period['HOME_PLAYER4_ID'] = period_starters['home_player_ids'][3]
    period['HOME_PLAYER5_ID'] = period_starters['home_player_ids'][4]
    period['AWAY_PLAYER1_ID'] = period_starters['away_player_ids'][0]
    period['AWAY_PLAYER2_ID'] = period_starters['away_player_ids'][1]
    period['AWAY_PLAYER3_ID'] = period_starters['away_player_ids'][2]
    period['AWAY_PLAYER4_ID'] = period_starters['away_player_ids'][3]
    period['AWAY_PLAYER5_ID'] = period_starters['away_player_ids'][4]
    
    # get index for all substitutions and for each one sub in and out appropriate players
    subs = period[period['EVENTMSGTYPE'] == 8].index.tolist()
    end = len(period.index)
    for i in range(len(subs)):
        if period['HOME_PLAYER1_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['HOME_PLAYER1_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['HOME_PLAYER2_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['HOME_PLAYER2_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['HOME_PLAYER3_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['HOME_PLAYER3_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['HOME_PLAYER4_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['HOME_PLAYER4_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['HOME_PLAYER5_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['HOME_PLAYER5_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['AWAY_PLAYER1_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['AWAY_PLAYER1_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['AWAY_PLAYER2_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['AWAY_PLAYER2_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['AWAY_PLAYER3_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['AWAY_PLAYER3_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['AWAY_PLAYER4_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['AWAY_PLAYER4_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
        elif period['AWAY_PLAYER5_ID'][subs[i]] == period['PLAYER1_ID'][subs[i]]:
            period['AWAY_PLAYER5_ID'][subs[i]:end] = period['PLAYER2_ID'][subs[i]]
    return period

We can now run this code for all games. Note that it wrote it this way becuase I would run it for like 200 games at a time because it took a while. This is doing it for 2013 and saving it...

In [9]:
c=0

In [ ]:
cc=21400001+c
if c>0:
    pbp_with_lineups=pd.read_csv('pbp_with_lineups.csv')
for i in range(cc,21400021,1):
    print(i)
    game_id= "00" + str(i)
    pbp = getRawPbpForGame(game_id)
    if i == 21400001:
        pbp_with_lineups = pbp.groupby("PERIOD").apply(getPlayersOnFloorForPeriod)
    else:
        pbp_with_lineups1 = pbp.groupby("PERIOD").apply(getPlayersOnFloorForPeriod)
        pbp_with_lineups = pbp_with_lineups.append(pbp_with_lineups1)
        pbp_with_lineups.to_csv('pbp_with_lineups.csv',index=False,header=True)
        c=c+1

In [ ]:
pbp_with_lineups.to_csv('playbyplay/pbp_with_lineups_2013.csv',index=False,header=True)

And this is doing it for 2014...

In [10]:
c=0

In [ ]:
cc=21400001+c
if c>0:
    pbp_with_lineups=pd.read_csv('pbp_with_lineups.csv')
for i in range(cc,21300021,1):
    print(i)
    game_id= "00" + str(i)
    pbp = getRawPbpForGame(game_id)
    if i == 21300001:
        pbp_with_lineups = pbp.groupby("PERIOD").apply(getPlayersOnFloorForPeriod)
    else:
        pbp_with_lineups1 = pbp.groupby("PERIOD").apply(getPlayersOnFloorForPeriod)
        pbp_with_lineups = pbp_with_lineups.append(pbp_with_lineups1)
        pbp_with_lineups.to_csv('pbp_with_lineups.csv',index=False,header=True)
        c=c+1

In [ ]:
pbp_with_lineups.to_csv('playbyplay/pbp_with_lineups_2014.csv',index=False,header=True)

Yay! Now we have all the play by play data! However, it does not in its current form have all the data we want. Not every moment has score differential, only when a player scores. We want to change that, so we get it so that each moment now contains score differential. We can do this by just shifting the column down a spot, and when it contains a NaN overwrite it with the score above.

In [ ]:
pbp2014=pd.read_csv('playbyplay/pbp_with_lineups_2014.csv')
sort=pbp2014.sort(['GAME_ID','EVENTNUM'])
all2014['EVENTNUM']=all2014['GAME_EVENT_ID']
clean = sort.drop_duplicates(['GAME_ID','EVENTNUM'])
k=0
while (sum([(str(x) == 'nan') for x in clean.SCOREMARGIN])>0)&(k<50):
    k+=1
    print k
    clean['SCOREMARGIN']=[y if str(x) =='nan' else x for x,y in zip(list(clean.SCOREMARGIN),[0]+list(clean.SCOREMARGIN)[:-1])]
clean['SCOREMARGIN']=[0 if x=='TIE' else x for x in clean.SCOREMARGIN]

The other piece of information we want is the action of the moment before, to identify fastbreaks.

In [ ]:
clean['ACTION_BEFORE']=["nan"]+list(clean.EVENTMSGTYPE)[:-1]

All that remains is to merge it with the shot data we have. Luckily, this proves pretty easy as the shot chart data (and therefor our merged shot data) has a unique identifier (game ID and moment ID) that is shared by play by play. Therefor, we just merge on those two columns, only retaining moments that are a shot. We also want the score margin to reflect the score from the shooters point of view (currently it is from the home team's point of view) so we negate it where necessary. We then save our data

In [ ]:
try1=clean.merge(all2014,on=['GAME_ID','EVENTNUM'])
try1['SCOREMARGIN']=[-1*int(y) if x =='A' else y for x,y in zip(try1.LOCATION,try1.SCOREMARGIN)]
try1.to_csv('merged_shots/merged_shots_14.csv')

Now we can do the same for the 2013 data...

In [ ]:
pbp2013=pd.read_csv('playbyplay/pbp_with_lineups_2013.csv')
sort=pbp2013.sort(['GAME_ID','EVENTNUM'])
all2013['EVENTNUM']=all2013['GAME_EVENT_ID']
clean = sort.drop_duplicates(['GAME_ID','EVENTNUM'])
k=0
while (sum([(str(x) == 'nan') for x in clean.SCOREMARGIN])>0)&(k<50):
    k+=1
    print k
    clean['SCOREMARGIN']=[y if str(x) =='nan' else x for x,y in zip(list(clean.SCOREMARGIN),[0]+list(clean.SCOREMARGIN)[:-1])]
clean['ACTION_BEFORE']=["nan"]+list(clean.EVENTMSGTYPE)[:-1]
clean['SCOREMARGIN']=[0 if x=='TIE' else x for x in clean.SCOREMARGIN]
try1=clean.merge(all2013,on=['GAME_ID','EVENTNUM'])
try1['SCOREMARGIN']=[-1*int(y) if x =='A' else y for x,y in zip(try1.LOCATION,try1.SCOREMARGIN)]
try1.to_csv('merged_shots/merged_shots_13.csv')

And BOOM! We're all done! Now that we have the data that we need and merged it all, we can go onto more fun things, like our analysis!